In [2]:
import findspark

findspark.init()


In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext()

In [5]:
%%writefile example.txt
first line
second line
third line 
fourth line

Overwriting example.txt


In [6]:
textFile = sc.textFile('example.txt')

In [7]:
textFile

example.txt MapPartitionsRDD[1] at textFile at <unknown>:0

In [8]:
textFile.count()

4

In [9]:
textFile.first()

'first line'

In [10]:
textFile.collect()

['first line', 'second line', 'third line ', 'fourth line']

In [11]:
secfile = textFile.filter(lambda line: 'second' in line)

In [12]:
secfile.collect()

['second line']

In [13]:
secfile.count()

1

In [14]:
csvFile = sc.textFile('Sales_April_2019.csv')

In [15]:
datapoints = csvFile.flatMap(lambda line: line.split(','))

In [16]:
del datapoints

In [17]:
del secfile

In [19]:
new_textFile = sc.textFile('services.txt')

In [20]:
new_textFile.collect()

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [21]:
 clean = new_textFile.map(lambda line:line.split())

In [22]:
del clean

In [23]:
new_textFile.collect()

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [24]:
clean = new_textFile.map(lambda lst:lst[1:] if lst[0]=='#'else lst)

In [25]:
clean.collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [26]:
clean = clean.map(lambda line : line.split())

In [27]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [28]:
state_amnt = clean.map(lambda lst : (lst[3] , lst[-1]))

In [29]:
state_amnt.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [30]:
pairs = state_amnt.reduceByKey(lambda amnt1,amnt2 : amnt1 + amnt2)

In [31]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00750.00'),
 ('TX', '450.00200.00'),
 ('CA', '200.00500.00')]

In [32]:
pairs  = state_amnt.reduceByKey(lambda amnt1,amnt2 : float(amnt1) + float(amnt2))

In [33]:
pairs.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [34]:
#filter

In [35]:
filterd = pairs.filter(lambda x :not x[0]=='State')

In [36]:
filterd.collect()

[('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [37]:
#sort

In [38]:
st_amnt = filterd.sortBy(lambda amnt:amnt[1],ascending=False)

In [63]:
st_amnt.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]